<a href="https://colab.research.google.com/github/jerondeepak/Project-ML/blob/main/Deepak_Jeron_9_7_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**NFH and NFL prediction in given dataset  Using Autogluon TabularPredictor**


step 1 : *data preprocessing*  split given dataset into two  dataset  with occurs NFH and NFL into two dataset and Normalize them .

step 2: *HPO* , using eval_metric = 'mean_squared_error'  because , its a regression type . then tune the parameters

step 3: display summary

step 4: *Model ensembling with stacking/bagging*

step 5: load model

step 6: *Model distillation* 


**NFH**
Fitting model: ExtraTreesMSE 
	-2974.5727	 = Validation mean_squared_error score


**NFL**

Fitting model: RandomForestMSE
	-9403.1067	 = Validation mean_squared_error 


In [1]:
# Uninstall mkl for faster neural-network training time
!pip uninstall -y mkl
# Upgrade pip to ensure the latest package versions are available
!pip install -U pip
# Upgrade setuptools to be compatible with namespace packages
!pip install -U setuptools wheel
!pip install -U "mxnet<2.0.0"
# Install autogluon (Tutorial based on autogluon==0.1.0)
!pip install autogluon
# Upgrade ipykernel (Necessary for Colab)

!pip install -U ipykernel

In [1]:

#import neccessary modules
import pandas as pd
import numpy as np
import autogluon.core as ag
from autogluon.tabular import TabularDataset, TabularPredictor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


DataSet = pd.read_csv('csv_nfl_data.csv')
DataSet.head()

with open('csv_nfl_data.csv','r') as f:
    data = f.read().split('\n')
data[1]=data[0]+data[1]
del data[0]
with open('changed_dataset.csv','w') as f:
    for i in data:
        f.write(i+'\n')


dataset = pd.read_csv('changed_dataset.csv')

dataset.head()

dataset.info()

dataset.drop("Date(IST)",inplace=True,axis=1)
dataset.isnull().sum()

null_list = ['FIIB','FIIS','FIIN','DIIB','DIIS','DIIN','August','December']
for i in null_list:
    dataset[i]=dataset[i].fillna(dataset[i].mean())

dataset.isnull().sum()

#with NFH and without NFL dataset 
dataset_NFH = dataset.drop('NFL',axis=1)
#with NFL and without NFH dataset
dataset_NFL = dataset.drop('NFH',axis=1)

def Train_Test_Split(dataset):
    train, test = train_test_split(dataset , train_size=0.7, test_size=0.3)
    return (train,test)

#NFH dataset split into train and test
train_NFH, test_NFH = Train_Test_Split(dataset_NFH)
#NFL dataset split into train and test
train_NFL, test_NFL = Train_Test_Split(dataset_NFL)

train_NFH.to_csv('train_NFH.csv', index=False)
test_NFH.to_csv('test_NFH.csv',index=False)
train_NFL.to_csv('train_NFL.csv',index=False)
test_NFL.to_csv('test_NFL.csv',index=False)




###########################################  NFH  ######################################
train_data = TabularDataset('train_NFH.csv')
subsample_size = 200  # subsample subset of data for faster demo, try setting this to much larger values
train_data = train_data.sample(n=subsample_size, random_state=0,replace=True)

label = 'NFH'

new_data = TabularDataset('test_NFH.csv')
test_data = new_data[75:].copy()  # this should be separate data in your applications
y_test = test_data[label]
test_data_nolabel = test_data.drop(columns=[label])  # delete label column
val_data = new_data[:75].copy()

sc=MinMaxScaler


display(train_data)

display(val_data)


metric = 'mean_squared_error' # we specify eval-metric just for demo (unnecessary as it's the default)

import autogluon.core as ag

nn_options = {  # specifies non-default hyperparameter values for neural network models
    'num_epochs': 10,  # number of training epochs (controls training time of NN models)
    'learning_rate': ag.space.Real(1e-4, 1e-2, default=5e-4, log=True),  # learning rate used in training (real-valued hyperparameter searched on log-scale)
    'activation': ag.space.Categorical('relu', 'softrelu', 'tanh'),  # activation function used in NN (categorical hyperparameter, default = first entry)
    'layers': ag.space.Categorical([10], [10], [20, 10], [30, 20, 10]),  # each choice for categorical hyperparameter 'layers' corresponds to list of sizes for each NN layer to use
    'dropout_prob': ag.space.Real(0.0, 0.5, default=0.1),  # dropout probability (real-valued hyperparameter)
}

gbm_options = {  # specifies non-default hyperparameter values for lightGBM gradient boosted trees
    'num_boost_round': 100,  # number of boosting rounds (controls training time of GBM models)
    'num_leaves': ag.space.Int(lower=26, upper=66, default=36),  # number of leaves in trees (integer hyperparameter)
}

hyperparameters = {  # hyperparameters of each model type
                   'GBM': gbm_options,
                   'NN': nn_options,  # NOTE: comment this line out if you get errors on Mac OSX
              
                  }  # When these keys are missing from hyperparameters dict, no models of that type are trained

time_limit = 2*60  # train various models for ~2 min
num_trials = 5  # try at most 5 different hyperparameter configurations for each type of model
search_strategy = 'auto'  # to tune hyperparameters using Bayesian optimization routine with a local scheduler

hyperparameter_tune_kwargs = {  # HPO is not performed unless hyperparameter_tune_kwargs is specified
    'num_trials': num_trials,
    'scheduler' : 'local',
    'searcher': search_strategy,
}



predictor = TabularPredictor(label=label, eval_metric=metric).fit(
    train_data, tuning_data=val_data, time_limit=time_limit,
    hyperparameters=hyperparameters, hyperparameter_tune_kwargs=hyperparameter_tune_kwargs,
)


y_pred = predictor.predict(test_data_nolabel)
print("Predictions:  ", list(y_pred)[:5])
perf = predictor.evaluate(test_data, auxiliary_metrics=False)

results = predictor.fit_summary() #Summary


#Model ensembling with stacking/bagging
predictor = TabularPredictor(label=label, eval_metric=metric).fit(train_data,
    num_bag_folds=5, num_bag_sets=1, num_stack_levels=1,
    hyperparameters = {'NN': {'num_epochs': 2}, 'GBM': {'num_boost_round': 20}},  # last  argument is just for quick demo here, omit it in real applications
)


save_path = 'ModelFolderNFH'  # folder where to store trained models

predictor = TabularPredictor(label=label, eval_metric=metric, path=save_path).fit(
    train_data, auto_stack=True,
    time_limit=30, hyperparameters={'NN': {'num_epochs': 2}, 'GBM': {'num_boost_round': 20}}  # last 2 arguments are for quick demo, omit them in real applications
)


#Prediction options (inference) NFH model
predictor = TabularPredictor.load(save_path)

datapoint = test_data_nolabel.iloc[[0]]  # Note: .iloc[0] won't work because it returns pandas Series instead of DataFrame
print(datapoint)
predictor.predict(datapoint)

predictor.predict_proba(datapoint)  # returns a DataFrame that shows which probability corresponds to which class

predictor.get_model_best()

predictor.leaderboard(test_data, silent=True)

predictor.leaderboard(extra_info=True, silent=True)

predictor.leaderboard(test_data, extra_metrics=['root_mean_squared_error', 'mean_squared_error', 'mean_absolute_error', 'median_absolute_error', 'r2'], silent=True)
i = 0  # index of model to use
model_to_use = predictor.get_model_names()[i]
model_pred = predictor.predict(datapoint, model=model_to_use)
print("Prediction from %s model: %s" % (model_to_use, model_pred.iloc[0]))


all_models = predictor.get_model_names()
model_to_use = all_models[i]
specific_model = predictor._trainer.load_model(model_to_use)

# Objects defined below are dicts of various information (not printed here as they are quite large):
model_info = specific_model.get_info()
predictor_information = predictor.info()

y_pred_proba = predictor.predict_proba(test_data_nolabel)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred_proba)

perf = predictor.evaluate(test_data)

predictor.feature_importance(test_data)

predictor.persist_models()

num_test = 20
preds = np.array(['']*num_test, dtype='object')
for i in range(num_test):
    datapoint = test_data_nolabel.iloc[[i]]
    pred_numpy = predictor.predict(datapoint, as_pandas=False)
    preds[i] = pred_numpy[0]

perf = predictor.evaluate_predictions(y_test[:num_test], preds, auxiliary_metrics=True)
print("Predictions: ", preds)

predictor.unpersist_models()

#Using smaller ensemble or faster model for prediction
additional_ensembles = predictor.fit_weighted_ensemble(expand_pareto_frontier=True)
print("Alternative ensembles you can use for prediction:", additional_ensembles)

predictor.leaderboard(only_pareto_frontier=True, silent=True)
model_for_prediction = additional_ensembles[0]
predictions = predictor.predict(test_data, model=model_for_prediction)
predictor.delete_models(models_to_delete=additional_ensembles, dry_run=False)  # delete these extra models so they don't affect rest of tutorial

#Collapsing bagged ensembles via refit_full
refit_model_map = predictor.refit_full()
print("Name of each refit-full model corresponding to a previous bagged ensemble:")
print(refit_model_map)
predictor.leaderboard(test_data, silent=True)

#ModelDistilation
student_models = predictor.distill(time_limit=30)  # specify much longer time limit in real applications
print(student_models)
preds_student = predictor.predict(test_data_nolabel, model=student_models[0])
print(f"predictions from {student_models[0]}:", list(preds_student)[:5])
predictor.leaderboard(test_data)

#Faster presets or hyperparameters
presets = ['good_quality_faster_inference_only_refit', 'optimize_for_deployment']
predictor_light = TabularPredictor(label=label, eval_metric=metric).fit(train_data, presets=presets, time_limit=30)
predictor_light = TabularPredictor(label=label, eval_metric=metric).fit(train_data, hyperparameters='very_light', time_limit=30)

excluded_model_types = ['KNN', 'NN', 'custom']
predictor_light = TabularPredictor(label=label, eval_metric=metric).fit(train_data, excluded_model_types=excluded_model_types, time_limit=30)




#########################################   NFL   ########################################
train_data = TabularDataset('train_NFL.csv')
subsample_size = 200  # subsample subset of data for faster demo, try setting this to much larger values
train_data = train_data.sample(n=subsample_size, random_state=0,replace=True)

label = 'NFL'

new_data = TabularDataset('test_NFL.csv')
test_data = new_data[75:].copy()  # this should be separate data in your applications
y_test = test_data[label]
test_data_nolabel = test_data.drop(columns=[label])  # delete label column
val_data = new_data[:75].copy()




display(train_data)

display(val_data)


metric = 'mean_squared_error' # we specify eval-metric just for demo (unnecessary as it's the default)

import autogluon.core as ag

nn_options = {  # specifies non-default hyperparameter values for neural network models
    'num_epochs': 10,  # number of training epochs (controls training time of NN models)
    'learning_rate': ag.space.Real(1e-4, 1e-2, default=5e-4, log=True),  # learning rate used in training (real-valued hyperparameter searched on log-scale)
    'activation': ag.space.Categorical('relu', 'softrelu', 'tanh'),  # activation function used in NN (categorical hyperparameter, default = first entry)
    'layers': ag.space.Categorical([10], [10], [20, 10], [30, 20, 10]),  # each choice for categorical hyperparameter 'layers' corresponds to list of sizes for each NN layer to use
    'dropout_prob': ag.space.Real(0.0, 0.5, default=0.1),  # dropout probability (real-valued hyperparameter)
}

gbm_options = {  # specifies non-default hyperparameter values for lightGBM gradient boosted trees
    'num_boost_round': 100,  # number of boosting rounds (controls training time of GBM models)
    'num_leaves': ag.space.Int(lower=26, upper=66, default=36),  # number of leaves in trees (integer hyperparameter)
}

hyperparameters = {  # hyperparameters of each model type
                   'GBM': gbm_options,
                   'NN': nn_options,  # NOTE: comment this line out if you get errors on Mac OSX
              
                  }  # When these keys are missing from hyperparameters dict, no models of that type are trained

time_limit = 2*60  # train various models for ~2 min
num_trials = 5  # try at most 5 different hyperparameter configurations for each type of model
search_strategy = 'auto'  # to tune hyperparameters using Bayesian optimization routine with a local scheduler

hyperparameter_tune_kwargs = {  # HPO is not performed unless hyperparameter_tune_kwargs is specified
    'num_trials': num_trials,
    'scheduler' : 'local',
    'searcher': search_strategy,
}



predictor = TabularPredictor(label=label, eval_metric=metric).fit(
    train_data, tuning_data=val_data, time_limit=time_limit,
    hyperparameters=hyperparameters, hyperparameter_tune_kwargs=hyperparameter_tune_kwargs,
)


y_pred = predictor.predict(test_data_nolabel)
print("Predictions:  ", list(y_pred)[:5])
perf = predictor.evaluate(test_data, auxiliary_metrics=False)

results = predictor.fit_summary() #Summary


#Model ensembling with stacking/bagging
predictor = TabularPredictor(label=label, eval_metric=metric).fit(train_data,
    num_bag_folds=5, num_bag_sets=1, num_stack_levels=1,
    hyperparameters = {'NN': {'num_epochs': 2}, 'GBM': {'num_boost_round': 20}},  # last  argument is just for quick demo here, omit it in real applications
)


save_path = 'ModelFolderNFL'  # folder where to store trained models

predictor = TabularPredictor(label=label, eval_metric=metric, path=save_path).fit(
    train_data, auto_stack=True,
    time_limit=30, hyperparameters={'NN': {'num_epochs': 2}, 'GBM': {'num_boost_round': 20}}  # last 2 arguments are for quick demo, omit them in real applications
)


#Prediction options (inference) NFH model
predictor = TabularPredictor.load(save_path)

datapoint = test_data_nolabel.iloc[[0]]  # Note: .iloc[0] won't work because it returns pandas Series instead of DataFrame
print(datapoint)
predictor.predict(datapoint)

predictor.predict_proba(datapoint)  # returns a DataFrame that shows which probability corresponds to which class

predictor.get_model_best()

predictor.leaderboard(test_data, silent=True)

predictor.leaderboard(extra_info=True, silent=True)

predictor.leaderboard(test_data, extra_metrics=['root_mean_squared_error', 'mean_squared_error', 'mean_absolute_error', 'median_absolute_error', 'r2'], silent=True)
i = 0  # index of model to use
model_to_use = predictor.get_model_names()[i]
model_pred = predictor.predict(datapoint, model=model_to_use)
print("Prediction from %s model: %s" % (model_to_use, model_pred.iloc[0]))


all_models = predictor.get_model_names()
model_to_use = all_models[i]
specific_model = predictor._trainer.load_model(model_to_use)

# Objects defined below are dicts of various information (not printed here as they are quite large):
model_info = specific_model.get_info()
predictor_information = predictor.info()

y_pred_proba = predictor.predict_proba(test_data_nolabel)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred_proba)

perf = predictor.evaluate(test_data)

predictor.feature_importance(test_data)

predictor.persist_models()

num_test = 20
preds = np.array(['']*num_test, dtype='object')
for i in range(num_test):
    datapoint = test_data_nolabel.iloc[[i]]
    pred_numpy = predictor.predict(datapoint, as_pandas=False)
    preds[i] = pred_numpy[0]

perf = predictor.evaluate_predictions(y_test[:num_test], preds, auxiliary_metrics=True)
print("Predictions: ", preds)

predictor.unpersist_models()

#Using smaller ensemble or faster model for prediction
additional_ensembles = predictor.fit_weighted_ensemble(expand_pareto_frontier=True)
print("Alternative ensembles you can use for prediction:", additional_ensembles)

predictor.leaderboard(only_pareto_frontier=True, silent=True)
model_for_prediction = additional_ensembles[0]
predictions = predictor.predict(test_data, model=model_for_prediction)
predictor.delete_models(models_to_delete=additional_ensembles, dry_run=False)  # delete these extra models so they don't affect rest of tutorial

#Collapsing bagged ensembles via refit_full
refit_model_map = predictor.refit_full()
print("Name of each refit-full model corresponding to a previous bagged ensemble:")
print(refit_model_map)
predictor.leaderboard(test_data, silent=True)

#ModelDistillation
student_models = predictor.distill(time_limit=30)  # specify much longer time limit in real applications
print(student_models)
preds_student = predictor.predict(test_data_nolabel, model=student_models[0])
print(f"predictions from {student_models[0]}:", list(preds_student)[:5])
predictor.leaderboard(test_data)

#Faster presets or hyperparameters
presets = ['good_quality_faster_inference_only_refit', 'optimize_for_deployment']
predictor_light = TabularPredictor(label=label, eval_metric=metric).fit(train_data, presets=presets, time_limit=30)
predictor_light = TabularPredictor(label=label, eval_metric=metric).fit(train_data, hyperparameters='very_light', time_limit=30)

excluded_model_types = ['KNN', 'NN', 'custom']
predictor_light = TabularPredictor(label=label, eval_metric=metric).fit(train_data, excluded_model_types=excluded_model_types, time_limit=30)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 358 entries, 0 to 357
Data columns (total 22 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   nfl_data   358 non-null    int64  
 1   Date(IST)  358 non-null    object 
 2   NFO        358 non-null    float64
 3   NFH        358 non-null    float64
 4   NFL        358 non-null    float64
 5   NFC        358 non-null    float64
 6   FIIB       357 non-null    float64
 7   FIIS       357 non-null    float64
 8   FIIN       357 non-null    float64
 9   DIIB       357 non-null    float64
 10  DIIS       357 non-null    float64
 11  DIIN       357 non-null    float64
 12  August     356 non-null    float64
 13  December   356 non-null    float64
 14  CAD        358 non-null    float64
 15  DAD        358 non-null    float64
 16  DOD        358 non-null    float64
 17  NDAD       358 non-null    float64
 18  Currey     358 non-null    float64
 19  Flow       358 non-null    float64
 20  Shine     

,nfl_data,NFO,NFH,NFC,FIIB,FIIS,FIIN,DIIB,DIIS,DIIN,August,December,CAD,DAD,DOD,NDAD,Currey,Flow,Shine,Vega
172,29,12131.397886,12165.472693,12100.911068,5071.12,5022.31,48.81,3795.48,3456.29,339.19,722.0,1109.0,0.10,0.2000,0.59,-0.74,0.0845,0.4492,-0.03,-1.440001
47,90,8868.805879,9024.544308,8939.041940,6102.70,6361.43,-258.73,3602.92,3201.14,401.78,1049.0,721.0,-0.07,-0.3360,-0.36,-0.72,0.1155,0.1793,-0.99,1.540001
117,12,12392.260613,12418.086057,12390.792053,6609.58,6345.32,264.26,3615.43,4115.60,-500.17,946.0,886.0,-0.10,-0.1400,0.86,1.00,0.1145,0.0681,-0.14,-0.219999
192,120,10477.183228,10574.081442,10540.191523,4111.43,3763.08,348.35,4205.92,3942.45,263.47,1160.0,771.0,0.10,0.4950,0.86,1.02,-0.1790,0.1416,0.34,0.260000
67,310,14425.143888,14448.520143,14288.410144,9619.52,10529.08,-909.56,5737.09,4887.11,849.98,1136.0,782.0,0.16,0.1465,-1.55,0.04,-0.3100,-0.2600,0.01,0.029999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30,273,15198.939630,15237.382613,15077.552219,9454.13,10347.38,-893.25,4804.05,5723.93,-919.88,569.0,1395.0,1.02,-0.0490,2.09,-2.81,-0.0295,0.6418,0.75,1.400002
24,344,15821.507577,15862.941950,15819.850213,4339.27,4842.78,-503.51,4801.07,4256.81,544.26,899.0,1138.0,0.05,0.0141,-1.04,2.21,-0.0910,0.0700,-0.08,0.740000
125,173,11595.460247,11609.162107,11538.238776,5230.50,5770.31,-539.81,4333.23,4851.18,-517.95,294.0,1639.0,-1.73,-1.3015,-2.46,-0.40,0.0020,-0.3487,-0.70,1.950001
2,249,14689.155319,14730.963589,14666.101429,10175.47,9098.85,1076.62,5772.80,5960.90,-188.10,912.0,1022.0,0.23,0.0968,-0.38,-0.36,0.0505,0.1800,0.08,1.040001


,nfl_data,NFO,NFH,NFC,FIIB,FIIS,FIIN,DIIB,DIIS,DIIN,August,December,CAD,DAD,DOD,NDAD,Currey,Flow,Shine,Vega
0,170,11503.579133,11544.444268,11501.759339,4980.880000,4716.220000,264.660000,4259.670000,4471.880000,-212.210000,966.0,864.0,0.23,0.4100,-1.37,-0.86,-0.0580,0.4786,0.00,0.450001
1,107,9837.276863,9983.324733,9873.176784,4608.090000,6086.610000,-1478.520000,4742.480000,3580.970000,1161.510000,847.0,1036.0,1.26,0.4890,0.27,1.16,0.2560,0.2257,-0.36,-0.730003
2,300,14773.489712,14852.499486,14823.579735,4825.290000,5918.040000,-1092.750000,4518.840000,4102.250000,416.590000,1158.0,774.0,-0.06,0.4611,0.00,1.81,0.1730,-0.4400,0.51,0.790001
3,134,11244.893725,11316.991796,11280.792472,7666.212493,7402.904678,263.307731,4715.688011,4785.561345,-69.873193,684.0,1190.0,-0.62,-0.2050,-0.21,-0.85,-0.0514,0.0119,0.45,-0.240000
4,221,13124.071523,13168.988803,13108.175126,9295.460000,8938.110000,357.350000,3377.190000,5013.160000,-1635.970000,1096.0,796.0,0.64,-0.0250,-0.33,-0.55,0.2151,0.1487,0.13,0.400000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,289,15065.660637,15083.937729,14927.645682,9973.210000,7347.390000,2625.820000,5874.440000,6436.590000,-562.150000,382.0,1582.0,0.43,0.3800,2.83,-1.45,0.0380,-0.0600,0.42,-0.560001
71,350,15589.111367,15770.988848,15728.427733,6488.060000,7516.000000,-1027.940000,5099.010000,4796.560000,302.450000,1212.0,792.0,-0.04,0.0500,0.39,-0.08,0.1535,0.0650,-0.07,-1.230000
72,320,14683.806797,14732.337352,14691.893652,7159.990000,8302.740000,-1142.750000,6414.230000,4946.140000,1468.090000,1097.0,834.0,0.08,0.6434,0.00,0.98,-0.3480,-0.0190,0.37,-1.699999
73,349,15751.096068,15790.801125,15704.512166,4675.480000,5920.190000,-1244.710000,4028.310000,3890.220000,138.090000,1255.0,770.0,1.24,0.5380,1.13,2.98,0.0470,0.3700,0.40,-2.810001


No path specified. Models will be saved in: "AutogluonModels/ag-20210715_083722/"
Beginning AutoGluon training ... Time limit = 120s
AutoGluon will save models to "AutogluonModels/ag-20210715_083722/"
AutoGluon Version:  0.2.0
Train Data Rows:    200
Train Data Columns: 19
Tuning Data Rows:    75
Tuning Data Columns: 19
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (15917.532072521899, 7988.390033328879, 12288.70726, 2106.16103)
	If 'regression' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    12719.08 MB
	Train Data (Original)  Memory Usage: 0.04 MB (0.0% of available memory)
	Inferr

Fitted model: LightGBM/T0 ...
	-27403.7762	 = Validation mean_squared_error score
	0.17s	 = Training runtime
	0.0s	 = Validation runtime
Fitted model: LightGBM/T1 ...
	-54939.3074	 = Validation mean_squared_error score
	0.16s	 = Training runtime
	0.0s	 = Validation runtime
Fitted model: LightGBM/T2 ...
	-27279.7499	 = Validation mean_squared_error score
	0.22s	 = Training runtime
	0.0s	 = Validation runtime
Fitted model: LightGBM/T3 ...
	-93986.651	 = Validation mean_squared_error score
	0.21s	 = Training runtime
	0.0s	 = Validation runtime
Fitted model: LightGBM/T4 ...
	-33199.9534	 = Validation mean_squared_error score
	0.18s	 = Training runtime
	0.0s	 = Validation runtime
Hyperparameter tuning model: NeuralNetMXNet ...


...

	-64970.9671	 = Validation mean_squared_error score
	0.51s	 = Training runtime
	0.01s	 = Validation runtime
Fitting model: CatBoost ... Training model for up to 26.71s of the 26.71s of remaining time.
	-78609.711	 = Validation mean_squared_error score
	1.8s	 = Training runtime
	0.0s	 = Validation runtime
Fitting model: NeuralNetFastAI ... Training model for up to 24.89s of the 24.88s of remaining time.
	-136651061.5378	 = Validation mean_squared_error score
	2.52s	 = Training runtime
	0.02s	 = Validation runtime
Fitting model: XGBoost ... Training model for up to 22.33s of the 22.32s of remaining time.
	-4573.7608	 = Validation mean_squared_error score
	0.29s	 = Training runtime
	0.0s	 = Validation runtime
Fitting model: NeuralNetMXNet ... Training model for up to 21.99s of the 21.99s of remaining time.
	-126747.6301	 = Validation mean_squared_error score
	2.86s	 = Training runtime
	0.17s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ... Training model for up to 29.85s of th

[1000]	train_set's l2: 2900.52	valid_set's l2: 66197.3


	-64970.9671	 = Validation mean_squared_error score
	0.55s	 = Training runtime
	0.01s	 = Validation runtime
Fitting model: LightGBM ... Training model for up to 29.19s of the 29.17s of remaining time.


[1000]	train_set's l2: 27.7168	valid_set's l2: 33637.1
[2000]	train_set's l2: 0.0305429	valid_set's l2: 33239
[3000]	train_set's l2: 5.02656e-05	valid_set's l2: 33235.3
[4000]	train_set's l2: 5.70207e-07	valid_set's l2: 33234.9
[5000]	train_set's l2: 3.74396e-08	valid_set's l2: 33234.9
[6000]	train_set's l2: 2.44957e-09	valid_set's l2: 33234.9


	-33234.8673	 = Validation mean_squared_error score
	2.2s	 = Training runtime
	0.02s	 = Validation runtime
Fitting model: RandomForestMSE ... Training model for up to 26.62s of the 26.6s of remaining time.
	-9403.1067	 = Validation mean_squared_error score
	0.74s	 = Training runtime
	0.1s	 = Validation runtime
Fitting model: CatBoost ... Training model for up to 25.74s of the 25.72s of remaining time.
	-78609.711	 = Validation mean_squared_error score
	1.82s	 = Training runtime
	0.0s	 = Validation runtime
Fitting model: ExtraTreesMSE ... Training model for up to 23.91s of the 23.9s of remaining time.
	-6496.9983	 = Validation mean_squared_error score
	0.66s	 = Training runtime
	0.1s	 = Validation runtime
Fitting model: NeuralNetFastAI ... Training model for up to 23.12s of the 23.1s of remaining time.
	-136655931.0981	 = Validation mean_squared_error score
	2.75s	 = Training runtime
	0.03s	 = Validation runtime
Fitting model: XGBoost ... Training model for up to 20.32s of the 20.3s of 